**------------------------------------------------------------------------------------------------------------------------------------------------------**

k-fold cross-validation

**Input: "Food" Embeddings**

**Evaluates Model by calculating MAP, MRR, and RR@[5, 10]**

**Output: Metrics**

**------------------------------------------------------------------------------------------------------------------------------------------------------**

# Libraries

In [1]:
import pandas as pd
import numpy as np
import random
import ast
from gensim.models import KeyedVectors
from gensim.test.utils import datapath
import warnings
warnings.simplefilter("ignore")

C:\Users\julie\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [11]:
#vModel = 'GCN'
vModel = 'GraphSAGE'
#vModel = 'GAT'

In [12]:
k_fold = 10

**Get Food Categories**

In [13]:
df = pd.read_csv('../Input Data/food_category.csv')
df['NDB_No'] = df['NDB_No'].astype(str).str.rjust(5,'0')

In [14]:
food2cat = {}
food_label_map = {}
food_id_map = {}
for i, row in df.iterrows():
    uri = 'http://idea.rpi.edu/heals/kb/usda#' + row['NDB_No']
    label = row['Long_Desc'].strip()
    cat = row['FdGrp_Desc']
    food2cat[uri] = cat
    food_label_map[uri] = label
    food_id_map[label] = uri

**Utils to Evaluate Model**

In [15]:
def get_simscore_ingrank_category_multisamerank(query_food, subs):
    # return rank based on KG embedding + foods' category
    TOPK = len(list(word_vectors.index_to_key))
    mostSimilar = word_vectors.most_similar(query_food, topn=TOPK)
    i = 0
    for e, sim in mostSimilar:
        if e not in food2cat or food2cat[subs] != food2cat[e]: continue
        if e == subs:
            return i+1 
        i+=1
    return TOPK

def evaluate_model(scraped_subs_dict):
    rank_scores = []
    ave_p = []
    in_top_5 = 0
    in_top_10 = 0
    print('number of ings: ',len(scraped_subs_dict.keys()))

    for query_food in sorted(scraped_subs_dict.keys()):
        relevant_ranks = []
        min_rank = 9999999999999999
        for subs in scraped_subs_dict[query_food]:
            rank = get_simscore_ingrank_category_multisamerank(query_food, subs)   
            relevant_ranks.append(rank)
            if rank < min_rank:
                min_rank = rank
        rank = min_rank
        rank_scores.append(1.0/rank)
        if min_rank <= 5:
            in_top_5 += 1
        if min_rank <= 10:
            in_top_10 += 1
        precisions = []
        for rank in relevant_ranks:
            good_docs = len([r for r in relevant_ranks if r <= rank])
            precisions.append(good_docs/rank)
        if len(precisions) == 0:
            precisions = [0]
        ave_p.append(np.mean(precisions))
    print(f'MAP: {(np.mean(ave_p)):.3f}')
    print(f'MRR: {(np.mean(rank_scores)):.3f}')
    print(f'RR@5: {(in_top_5/len(scraped_subs_dict.keys())):.3f}')
    print(f'RR@10: {(in_top_10/len(scraped_subs_dict.keys())):.3f}')
    return (np.mean(ave_p)), (np.mean(rank_scores)), (in_top_5/len(scraped_subs_dict.keys())), (in_top_10/len(scraped_subs_dict.keys()))

**Get Ground Truth**

In [17]:
subs_df = pd.read_csv('../Input Data/final_substitution.csv', sep=';')
subs_df['Food id'] = subs_df['Food id']
subs_df['Substitution id'] = subs_df['Substitution id']

In [18]:
def get_scraped_subs_dict(word_vectors):
    
    scraped_subs_dict = dict()

    for i,row in subs_df.iterrows():
        food = row['Food id']
        subs = row['Substitution id']
        if food in word_vectors and subs in word_vectors and food in food2cat and subs in food2cat:
            if food not in scraped_subs_dict:
                scraped_subs_dict[food]= set()

            scraped_subs_dict[food].add(subs)
        else:
            print (food, subs, 'not in embeddings')
            
    return scraped_subs_dict

**Get Evaluations**

In [19]:
mean_average_precision = []
mean_reciprocal_rank = []
recall_rate_5 = []
recall_rate_10 = []

for i in range(k_fold): 
    
    print('------------------------------------------------------------------')
    
    #Get Embeddings and Ground Truth
    word_vectors = KeyedVectors.load_word2vec_format(f'../Output/k_fold/{vModel}_{i}_food_embeddings.txt', binary=False)
    scraped_subs_dict = get_scraped_subs_dict(word_vectors)

    #Restrict Ground Truth to the Test Foods
    foods = pd.read_csv(f'../Output/k_fold/{vModel}_{i}_foods_2_test.csv')
    new_scraped_subs_dict = dict()
    for food in np.unique(np.array(foods['id'])):
        if food in scraped_subs_dict:
            new_scraped_subs_dict[food] = scraped_subs_dict[food]
        else:
            print (food, 'not in ground truth')

    map_, mrr_, rr5_, rr10_ = evaluate_model(new_scraped_subs_dict)
    mean_average_precision.append(map_)
    mean_reciprocal_rank.append(mrr_)
    recall_rate_5.append(rr5_)
    recall_rate_10.append(rr10_)

------------------------------------------------------------------
number of ings:  126
MAP: 0.261
MRR: 0.402
RR@5: 0.571
RR@10: 0.706
------------------------------------------------------------------
number of ings:  133
MAP: 0.287
MRR: 0.448
RR@5: 0.624
RR@10: 0.759
------------------------------------------------------------------
number of ings:  119
MAP: 0.316
MRR: 0.451
RR@5: 0.639
RR@10: 0.790
------------------------------------------------------------------
number of ings:  129
MAP: 0.279
MRR: 0.456
RR@5: 0.643
RR@10: 0.713
------------------------------------------------------------------
number of ings:  127
MAP: 0.284
MRR: 0.539
RR@5: 0.732
RR@10: 0.835
------------------------------------------------------------------
number of ings:  124
MAP: 0.281
MRR: 0.430
RR@5: 0.621
RR@10: 0.758
------------------------------------------------------------------
number of ings:  113
MAP: 0.310
MRR: 0.506
RR@5: 0.664
RR@10: 0.761
-------------------------------------------------------

In [20]:
round(np.array(mean_average_precision).mean(), 3)

0.279

In [21]:
round(np.array(mean_reciprocal_rank).mean(), 3)

0.451

In [22]:
round(np.array(recall_rate_5).mean(), 3)

0.624

In [23]:
round(np.array(recall_rate_10).mean(), 3)

0.746